In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!pip install git+https://github.com/qubvel/classification_models.git -q
!pip install -U efficientnet -q
!pip install tensorflow_addons -q

In [ ]:
import numpy as np
import pandas as pd
from tqdm.auto import tqdm
import random
import os
import re
import math
import sys
sys.path.append('/content/drive/MyDrive/TFNFNet')

tqdm.pandas()

import matplotlib.pyplot as plt
import cv2

from sklearn.model_selection import train_test_split, StratifiedKFold
from sklearn import metrics

import tensorflow as tf
import tensorflow_addons as tfa
import tensorflow_hub as hub

import efficientnet.tfkeras as efn
from classification_models.tfkeras import Classifiers
from nfnet import NFNet

import logging
logging.getLogger("tensorflow").setLevel(logging.WARNING)
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' 

def set_seed(seed = 0):
    '''Sets the seed of the entire notebook so results are the same every time we run.
    This is for REPRODUCIBILITY.'''
    tf.random.set_seed(seed)
    np.random.seed(seed)
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    
import warnings
warnings.filterwarnings('ignore')

In [ ]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  # TPU detection. No parameters necessary if TPU_NAME environment variable is set. On Kaggle this is always the case.
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() # default distribution strategy in Tensorflow. Works on CPU and single GPU.

print("REPLICAS: ", strategy.num_replicas_in_sync)

Running on TPU  grpc://10.32.102.114:8470
REPLICAS:  8


In [ ]:
EPOCHS = 15
FOLDS = 5
FOLD_TO_TRAIN = 0
OLD_DIM = 256
DIM = 512
SEED = 42
IMAGE_SIZE = [DIM, DIM]
AUTO = tf.data.experimental.AUTOTUNE
GCS_DS_PATH = "gs://kds-03e1a10468eed52cf1f7962028d75ca9e9ca67e4b3c30a4f640075f4"
LR = 1e-4
BATCH_SIZE = strategy.num_replicas_in_sync*16
PRETRAINED_MODEL = 'efficientnetv2-xl-21k'
STEPS_PER_EPOCH = 2508*(FOLDS-1)/BATCH_SIZE
TTA_STEPS = 5
NFNET_VARIANT = 'F1'
hub_url = f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/{PRETRAINED_MODEL}/feature-vector'
MODEL_GCS_PATH = 'gs://kds-fe3e67555fd7fb37eb13b8ccd52726e27768b43c6a31332c1a38a9e5'

In [ ]:
sample_sub = pd.read_csv('/content/drive/MyDrive/Kaggle/sample_submission.csv')

In [ ]:
sample_sub['path'] = sample_sub['id'].progress_apply(lambda x:  f"{GCS_DS_PATH}/test/{x[0]}/{x}.npy")

In [ ]:
def build_decoder(with_labels=True, target_size=(256, 256), ext='npy'):
    def decode(path):
        file_bytes = tf.io.read_file(path)
        if ext == 'npy':
            img = tf.io.decode_raw(file_bytes, tf.float16)
            img = img[64:]
            img = tf.reshape(img, [6, 273, 256])
            img = tf.concat([img[0,:,:], img[2,:,:], img[4,:,:]], axis=0)
            img = tf.stack([img, img, img], axis=-1)
            img = tf.cast(img, tf.float32) 
            img = tf.image.resize(img, target_size)
        else:
            if ext == 'png':
                img = tf.image.decode_png(file_bytes, channels=3)
            elif ext in ['jpg', 'jpeg']:
                img = tf.image.decode_jpeg(file_bytes, channels=3)
            else:
                raise ValueError("Image extension not supported")
            img = tf.cast(img, tf.float32) / 255.0
            img = tf.image.resize(img, target_size)

        return img
    
    def decode_with_labels(path, label):
        return decode(path), label
    
    return decode_with_labels if with_labels else decode


def build_augmenter(with_labels=True):
    def augment(img):
        img = tf.image.random_flip_left_right(img)
        img = tf.image.random_flip_up_down(img)
        return img
    
    def augment_with_labels(img, label):
        return augment(img), label
    
    return augment_with_labels if with_labels else augment


def build_dataset(paths, labels=None, bsize=128, cache=True,
                  decode_fn=None, augment_fn=None,
                  augment=True, repeat=True, shuffle=1024, 
                  cache_dir=""):
    if cache_dir != "" and cache is True:
        os.makedirs(cache_dir, exist_ok=True)
    
    if decode_fn is None:
        decode_fn = build_decoder(labels is not None)
    
    if augment_fn is None:
        augment_fn = build_augmenter(labels is not None)
    
    AUTO = tf.data.experimental.AUTOTUNE
    slices = paths if labels is None else (paths, labels)
    
    dset = tf.data.Dataset.from_tensor_slices(slices)
    dset = dset.map(decode_fn, num_parallel_calls=AUTO)
    dset = dset.cache(cache_dir) if cache else dset
    dset = dset.map(augment_fn, num_parallel_calls=AUTO) if augment else dset
    dset = dset.repeat() if repeat else dset
    dset = dset.shuffle(shuffle) if shuffle else dset
    dset = dset.batch(bsize).prefetch(AUTO)
    
    return dset

In [ ]:
def transform(image,label):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 1.* tf.random.normal([1], dtype='float32', seed=42)
    shr = tf.random.normal([1], dtype='float32', seed=42) 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32', seed=42)/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32', seed=42)/10.
    h_shift = 16. * tf.random.normal([1], dtype='float32', seed=42) 
    w_shift = 16. * tf.random.normal([1], dtype='float32', seed=42) 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = tf.keras.backend.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = tf.keras.backend.cast(idx2,dtype='int32')
    idx2 = tf.keras.backend.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3]),label

def transform_test(image):
    # input image - is one image of size [dim,dim,3] not a batch of [b,dim,dim,3]
    # output - image randomly rotated, sheared, zoomed, and shifted
    DIM = IMAGE_SIZE[0]
    XDIM = DIM%2 #fix for size 331
    
    rot = 1.* tf.random.normal([1], dtype='float32', seed=42)
    shr = tf.random.normal([1], dtype='float32', seed=42) 
    h_zoom = 1.0 + tf.random.normal([1], dtype='float32', seed=42)/10.
    w_zoom = 1.0 + tf.random.normal([1],dtype='float32', seed=42)/10.
    h_shift = 16. * tf.random.normal([1], dtype='float32', seed=42) 
    w_shift = 16. * tf.random.normal([1], dtype='float32', seed=42) 
  
    # GET TRANSFORMATION MATRIX
    m = get_mat(rot,shr,h_zoom,w_zoom,h_shift,w_shift) 

    # LIST DESTINATION PIXEL INDICES
    x = tf.repeat( tf.range(DIM//2,-DIM//2,-1), DIM )
    y = tf.tile( tf.range(-DIM//2,DIM//2),[DIM] )
    z = tf.ones([DIM*DIM],dtype='int32')
    idx = tf.stack( [x,y,z] )
    
    # ROTATE DESTINATION PIXELS ONTO ORIGIN PIXELS
    idx2 = tf.keras.backend.dot(m,tf.cast(idx,dtype='float32'))
    idx2 = tf.keras.backend.cast(idx2,dtype='int32')
    idx2 = tf.keras.backend.clip(idx2,-DIM//2+XDIM+1,DIM//2)
    
    # FIND ORIGIN PIXEL VALUES           
    idx3 = tf.stack( [DIM//2-idx2[0,], DIM//2-1+idx2[1,]] )
    d = tf.gather_nd(image,tf.transpose(idx3))
        
    return tf.reshape(d,[DIM,DIM,3])

In [ ]:
AUG_BATCH = BATCH_SIZE

def cutmix(image, label, PROBABILITY = 1.0):
    
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with cutmix applied
    DIM = IMAGE_SIZE[0]
    CLASSES = 1
    label = tf.cast(label, 'float32')
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO CUTMIX WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.int32)
        # CHOOSE RANDOM IMAGE TO CUTMIX WITH
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        # CHOOSE RANDOM LOCATION
        x = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        y = tf.cast( tf.random.uniform([],0,DIM),tf.int32)
        b = tf.random.uniform([],0,1) # this is beta dist with alpha=1.0
        WIDTH = tf.cast( DIM * tf.math.sqrt(1-b),tf.int32) * P
        ya = tf.math.maximum(0,y-WIDTH//2)
        yb = tf.math.minimum(DIM,y+WIDTH//2)
        xa = tf.math.maximum(0,x-WIDTH//2)
        xb = tf.math.minimum(DIM,x+WIDTH//2)
        # MAKE CUTMIX IMAGE
        one = image[j,ya:yb,0:xa,:]
        two = image[k,ya:yb,xa:xb,:]
        three = image[j,ya:yb,xb:DIM,:]
        middle = tf.concat([one,two,three],axis=1)
        img = tf.concat([image[j,0:ya,:,:],middle,image[j,yb:DIM,:,:]],axis=0)
        imgs.append(img)
        # MAKE CUTMIX LABEL
        a = tf.cast(WIDTH*WIDTH/DIM/DIM,tf.float32)

        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)
            
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH, DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH, 1))
    return image2,label2

def mixup(image, label, PROBABILITY = 1.0):
    # input image - is a batch of images of size [n,dim,dim,3] not a single image of [dim,dim,3]
    # output - a batch of images with mixup applied
    DIM = IMAGE_SIZE[0]
    CLASSES = 1
    label = tf.cast(label, 'float32')
    
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        # DO MIXUP WITH PROBABILITY DEFINED ABOVE
        P = tf.cast( tf.random.uniform([],0,1)<=PROBABILITY, tf.float32)
        # CHOOSE RANDOM
        k = tf.cast( tf.random.uniform([],0,AUG_BATCH),tf.int32)
        a = 0.5 #tf.random.uniform([],0,1)*P # this is beta dist with alpha=1.0
        # MAKE MIXUP IMAGE
        img1 = image[j,]
        img2 = image[k,]
        imgs.append((1-a)*img1 + a*img2)
        # MAKE CUTMIX LABEL

        lab1 = label[j,]
        lab2 = label[k,]
        labs.append((1-a)*lab1 + a*lab2)

    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image2 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label2 = tf.reshape(tf.stack(labs),(AUG_BATCH,1))
    return image2,label2

def transform_cutmix_mixup(image,label):
    # THIS FUNCTION APPLIES BOTH CUTMIX AND MIXUP
    DIM = IMAGE_SIZE[0]
    SWITCH = 0.5
    CUTMIX_PROB = 0.666
    MIXUP_PROB = 0.666
    label = tf.cast(label, 'float32')
    # FOR SWITCH PERCENT OF TIME WE DO CUTMIX AND (1-SWITCH) WE DO MIXUP
    image2, label2 = cutmix(image, label, CUTMIX_PROB)
    image3, label3 = mixup(image, label, MIXUP_PROB)
    imgs = []; labs = []
    for j in range(AUG_BATCH):
        P = tf.cast( tf.random.uniform([],0,1)<=SWITCH, tf.float32)
        imgs.append(P*image2[j,]+(1-P)*image3[j,])
        labs.append(P*label2[j,]+(1-P)*label3[j,])
    # RESHAPE HACK SO TPU COMPILER KNOWS SHAPE OF OUTPUT TENSOR (maybe use Python typing instead?)
    image4 = tf.reshape(tf.stack(imgs),(AUG_BATCH,DIM,DIM,3))
    label4 = tf.reshape(tf.stack(labs),(AUG_BATCH,1))
    return image4,tf.reshape(tf.cast(tf.math.round(label4), 'int64'), (-1,))

In [ ]:
LR_START = LR
LR_MAX = LR*4
LR_MIN = LR/10
LR_RAMPUP_EPOCHS = 5
LR_SUSTAIN_EPOCHS = 0
LR_EXP_DECAY = .8

def CustomSchedule(epoch):
    if epoch < LR_RAMPUP_EPOCHS:
        lr = (LR_MAX - LR_START) / LR_RAMPUP_EPOCHS * epoch + LR_START
    elif epoch < LR_RAMPUP_EPOCHS + LR_SUSTAIN_EPOCHS:
        lr = LR_MAX
    else:
        lr = (LR_MAX - LR_MIN) * LR_EXP_DECAY**(epoch - LR_RAMPUP_EPOCHS - LR_SUSTAIN_EPOCHS) + LR_MIN
    return lr

def ExponentialSchedule(epoch, lr):
    rate = DECAY_RATE
    return lr*tf.math.exp(-rate*(epoch)).numpy()


CustomCallback = tf.keras.callbacks.LearningRateScheduler(CustomSchedule, verbose = True)
EarlyStopping = tf.keras.callbacks.EarlyStopping(monitor='val_AUC', patience=10, mode='max', restore_best_weights=True)
ReduceOnPlateau = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_AUC', mode='max', min_lr=1e-8)

In [ ]:
paths = sample_sub['path'].values

In [ ]:
with strategy.scope():
    
    ############# B6 #############
    
    # model1 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B6/B6_mixup_0_V2.h5')
    # model2 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B6/B6_mixup_1_V2.h5')
    # model3 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B6/B6_mixup_2_V2.h5')
    # model4 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B6/B6_mixup_3_V2.h5')
    # model5 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B6/B6_mixup_4_V2.h5')
    
    ############# B7 #############
    
    # model6 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B7/B7_mixup_0_V2.h5')
    # model7 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B7/B7_mixup_1_V2.h5')
    # model8 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B7/B7_mixup_2_V2.h5')
    # model9 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B7/B7_mixup_3_V2.h5')
    # model10 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/EfficientNet-B7/B7_mixup_4_V2.h5')
    
#     ############# V2-m-21k #############
    
#     model11 = tf.keras.models.load_model('../input/efficientnetv2m21k/V2-m/efficientnetv2-m-21k_finetuned_0.h5', custom_objects={'KerasLayer': hub.KerasLayer})
#     model12 = tf.keras.models.load_model('../input/efficientnetv2m21k/V2-m/efficientnetv2-m-21k_finetuned_1.h5', custom_objects={'KerasLayer': hub.KerasLayer})
#     model13 = tf.keras.models.load_model('../input/efficientnetv2m21k/V2-m/efficientnetv2-m-21k_finetuned_2.h5', custom_objects={'KerasLayer': hub.KerasLayer})
#     model14 = tf.keras.models.load_model('../input/efficientnetv2m21k/V2-m/efficientnetv2-m-21k_finetuned_3.h5', custom_objects={'KerasLayer': hub.KerasLayer})
#     model15 = tf.keras.models.load_model('../input/efficientnetv2m21k/V2-m/efficientnetv2-m-21k_finetuned_4.h5', custom_objects={'KerasLayer': hub.KerasLayer})
    
    ############# V2-xl-21k #############
    
    model16 = tf.keras.models.load_model('../input/efficientnetv2xl21k/V2-xl/efficientnetv2-xl-21k_finetuned_0.h5', custom_objects={'KerasLayer': hub.KerasLayer})
    model17 = tf.keras.models.load_model('../input/efficientnetv2xl21k/V2-xl/efficientnetv2-xl-21k_finetuned_1.h5', custom_objects={'KerasLayer': hub.KerasLayer})
    model18 = tf.keras.models.load_model('../input/efficientnetv2xl21k/V2-xl/efficientnetv2-xl-21k_finetuned_2.h5', custom_objects={'KerasLayer': hub.KerasLayer})
    model19 = tf.keras.models.load_model('../input/efficientnetv2xl21k/V2-xl/efficientnetv2-xl-21k_finetuned_3.h5', custom_objects={'KerasLayer': hub.KerasLayer})
    model20 = tf.keras.models.load_model('../input/efficientnetv2xl21k/V2-xl/efficientnetv2-xl-21k_finetuned_4.h5', custom_objects={'KerasLayer': hub.KerasLayer})

    # save_locally = tf.saved_model.SaveOptions(experimental_io_device='/job:localhost')

#     ############# F0 #############
        
#     model21 = tf.keras.models.load_model('../input/nfnet-finetuned/NFNet_finetuned/F0/0_finetuned', options=save_locally)
#     model22 = tf.keras.models.load_model('../input/nfnet-finetuned/NFNet_finetuned/F0/1_finetuned', options=save_locally)
#     model23 = tf.keras.models.load_model('../input/nfnet-finetuned/NFNet_finetuned/F0/2_finetuned', options=save_locally)
#     model24 = tf.keras.models.load_model('../input/nfnet-finetuned/NFNet_finetuned/F0/3_finetuned', options=save_locally)
#     model25 = tf.keras.models.load_model('../input/nfnet-finetuned/NFNet_finetuned/F0/4_finetuned', options=save_locally)

    ############# F1 #############
       
    # model26 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F1/0_finetuned_V2', options=save_locally)
    # model27 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F1/1_finetuned_V2', options=save_locally)
    # model28 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F1/2_finetuned_V2', options=save_locally)
    # model29 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F1/3_finetuned_V2', options=save_locally)
    # model30 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F1/4_finetuned_V2', options=save_locally)

    ############# F2 #############
       
    # model31 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F2/0_finetuned_V2', options=save_locally)
    # model32 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F2/1_finetuned_V2', options=save_locally)
    # model33 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F2/2_finetuned_V2', options=save_locally)
    # model34 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F2/3_finetuned_V2', options=save_locally)
    # model35 = tf.keras.models.load_model('/content/drive/MyDrive/Kaggle/weights/Finetuned_weights/NFNet-F2/4_finetuned_V2', options=save_locally)

In [ ]:
decoder = build_decoder(with_labels=False, target_size=IMAGE_SIZE, ext='npy')
test_dataset = build_dataset(
    paths,
    bsize=BATCH_SIZE, 
    decode_fn=decoder,
    shuffle=False, 
    repeat=False, 
    augment=True
)

In [ ]:
predictions = []

for i in range(TTA_STEPS):
    print(f'Step: {i+1}')
    preds = np.mean([
        # model1.predict(test_dataset, verbose=1),
        # model2.predict(test_dataset, verbose=1),
        # model3.predict(test_dataset, verbose=1),
        # model4.predict(test_dataset, verbose=1),
        # model5.predict(test_dataset, verbose=1), 
        
        # model6.predict(test_dataset, verbose=1),
        # model7.predict(test_dataset, verbose=1),
        # model8.predict(test_dataset, verbose=1),
        # model9.predict(test_dataset, verbose=1),
        # model10.predict(test_dataset, verbose=1),
        
#         model11.predict(test_dataset, verbose=1),
#         model12.predict(test_dataset, verbose=1),
#         model13.predict(test_dataset, verbose=1),
#         model14.predict(test_dataset, verbose=1),
#         model15.predict(test_dataset, verbose=1),
        
#         model16.predict(test_dataset, verbose=1),
#         model17.predict(test_dataset, verbose=1),
#         model18.predict(test_dataset, verbose=1),
#         model19.predict(test_dataset, verbose=1),
#         model20.predict(test_dataset, verbose=1),
        
#         model21.predict(test_dataset, verbose=1),
#         model22.predict(test_dataset, verbose=1),
#         model23.predict(test_dataset, verbose=1),
#         model24.predict(test_dataset, verbose=1),
#         model25.predict(test_dataset, verbose=1),
        
        model26.predict(test_dataset, verbose=1),
        model27.predict(test_dataset, verbose=1),
        model28.predict(test_dataset, verbose=1),
        model29.predict(test_dataset, verbose=1),
        model30.predict(test_dataset, verbose=1),

        model31.predict(test_dataset, verbose=1),
        model32.predict(test_dataset, verbose=1),
        model33.predict(test_dataset, verbose=1),
        model34.predict(test_dataset, verbose=1),
        model35.predict(test_dataset, verbose=1)
        
    ], axis=0)
    predictions.append(preds)
    
final_predictions = np.mean(predictions, axis=0).reshape(-1,)

In [ ]:
sample_sub['target'] = final_predictions
sample_sub = sample_sub.drop(columns='path')
sample_sub.to_csv('submission.csv', index=False)

In [ ]:
sample_sub